input is getting scrambled somehow. Inserting aaa yields no change, while inserting ab has random output \

300 digits/ new char - implies they are encrypting each character individually and concatenating the resulting ciphertexts\
bingo. if a char 'a' is in position 0 of 1 message, they will get encrypted to the same ciphertext in another message. 
we can get the scrambled flag with this\
answer code is at the bottom

In [1]:
from math import floor, log2
from sympy import prime
import random

In [2]:

def lcm(a,b):
    return abs(a*b)//extended_gcd(a,b)[0]
ascii32bit = "abcdefghijklmnopqrstuvwxyz.,!?' "
def ascii32bit_encode(s: str):
    acc = 0
    i = 0
    for c in s:
        acc+=(ascii32bit.index(c) if c in ascii32bit else 31)<<i
        i+=5
    return acc
# edge case where last char has leading zeros, need to adjust bit shift offset accordingly
def ascii32bit_decode(s: int):
    acc = s
    ans = ""
    n = round(log2(s)) - 5
    n = n+(5-n%5) if n%5!=0 else n
    for i in range(n, -1, -5):
        temp = acc >> i
        temp2 = temp << i
        ans=ascii32bit[temp]+ans
        acc -= temp2
    return ans
def fast_mod_exp(b, exp, m):
    res = 1
    while exp > 1:
        if exp & 1:
            res = (res * b) % m
        b = b ** 2 % m
        exp >>= 1
    return (b * res) % m
# ported from https://stackoverflow.com/questions/2921406/calculate-primes-p-and-q-from-private-exponent-d-public-exponent-e-and-the
# given d e and n, find p and q
def pq_from_den(d, e, n):
    k = d*e-1
    if (k%2==0):
        r = k
        t = 0
        while (r%2==0):
            r = r >> 1
            t+=1
        success = False
        y = -1
        for i in range(1,101):
            g = -1
            g = random.randrange(0,n)
            y = fast_mod_exp(g,r,n)
            if y==1 or y==n-1:
                continue
            for j in range(1, t):
                x = fast_mod_exp(y,2,n)
                if x==1:
                    success = True
                    break
                if x==n-1:
                    continue
                y = x
            x = fast_mod_exp(y, 2, n)
            if (x==1):
                success=True
                break
        if (success):
            p = gcd(y-1, n)[0]
            q = n//p
            return p, q
        else:
            assert False, "not found"
    else:
        assert False, "not found"
def extended_gcd(a, b):
    r1 = b
    r2 = a
    u1 = 0
    v1 = 1
    u2 = 1
    v2 = 0
    while r2 != 0:
        q = r1//r2
        r3 = r1
        u3 = u1
        v3 = v1
        r1 = r2
        u1 = u2
        v1 = v2
        r2 = r3 - q * r2
        u2 = u3 - q * u2
        v2 = v3 - q * v2
    return (r1, u1, v1)

In [3]:
from Crypto.Util.number import getPrime, bytes_to_long, long_to_bytes
from sympy import root
# 1192354900000307982173917546576252824083567740688852229484566839716766920380689316509595685295718288175717414414697507525651278137995739559965856448833119705499538490407645073085513569018893352855204794403053119079213654064308064565417219976332055692488588172613348498604320897914485063514716280621229104503495016852972354482931330655045399627752944884481614759516494183193736138677256340238538218062082453751217565083526950282758370612481067241909156781957634648211075761470536522753673656908910832723349372663856925438465078736197253929860175787639543543754861720018983745918457193651025007131009571282877329825230117411908065125004645057254687207623270058524415962618980157934274580579500315439104196592265244947386513853281957941373050001062356560201081494665044849489465165814293064873712342701319481159151924782720946403360846794560941726650309908615517628482153163991037279085998724801324194816193175411080338959333761999780241315652361360702072580071444285511795058071407433928088469365832113168483915430993469107215832912488376104806532336017007884548137711208120661078207529432424416787567957084084919052455225851008673739022476094899696979967830214555383798296669101544249025535753800718975347585150259357394949650490565214056799345500140768040344602957256707695125972844465203815951968531068623903093080446331821064607186513890733105079382856148828308930348209641222268401246613925175222505024390715259386020460564842539726615518055442364786359014705045477349205932511125041510660530399936081539807715899941531383529823632938600029918266142365617931719144539137243943350967471049803166741239539787815947344467347551421064564996400676257921309754331361374087766398672456350753461018811771103232976920941122483421939966293539714638433321055465112699677653751424058751892237143170102828327478858837456813085765086875459288440840870697176516529189758380991459405300187680017311403983536279587807891656672347106088061751789197268087061661651089450625894970539721868987755711285166927244018987849591927584855945915353374357986515242583960030985482783805340816945167189794567699656104890325606077483299110172492320726961266711755843750444760130459512871372002911955853436207834229149316699596032570071970628300834019000756579025953730861212193518892269313666983166399743904461019336127012652134511049871971834035230582223069505866467796861549907441303976846553860391684329314817291573812641161660024370765881282564969492896248170131001349814613983092463391183782290670937981290093399132065982611587016299046084479532247761197385147324662830732119318265052987111916906308550984152957790570756899628442848345726447716839950660786260062698630091678911085897120183263857435307377474911599537828774042802152033630225788698969249974321656418917827584430774367325964670196426061868365436121747135449257957919817503850253283874987333969053643717240132750087151875932984325979305344806678388310944847483826232138999314446535762440702305074016297737166465824803702339342751540408123376715882426444883319595161564691719048549305881876059206585000934886925933491368130770932311151240965420429897008935212891157322881078184196864585423851873278155991170733607288952880423060068857429354152470016830890333468306108028808857408485252210049153910143795342114856550551380499731474094150153173475524208927780119957036710656558699836687104672183417621006982982908278556637563618237084710501504809693615729290651052295197291311648912592475478676439973858422647857447591626240147492541604405082526266404216651540918511059572717633788737158344066122248473611432319954662631257218753494620457601748981133763568661529937398058027522974949681024213282410163031175517768362011465109532059367941894333785768480263491549316139288872801224553927034747030775326982165020055971619522479489843902259436267095326427463839439600222209166445827592567534175434972417940052479092153278176454850147583444500630408810612133168445111088189894687305545795113430481953745541153502084672904135065066037448190249642770310431401725068292304537111410731751397933792652018322965214395039029156389570427967981932663406667408506339180366529156310707069833856598910791396022455596875097616742701974636464548184645231475510584283046714765245184876552270449619252163177563063292079978687131992062844229593788574226982280728081012896146799464873687586765552181111546093045728852914821389182099791498557873457110502330458463938105140676603930877519876100027816892793263080624477664147737890158546908409849212665025529068693748020894448276461264046272476970668697997607113148639662980673724715524356892661892265331160730296021038368451850418190688770617999028536253137231210844589291357550999208543624052501451434109667322520932455818273718596035051264064347345737212191699837721975643430601377085349139182193605311642965283979764949123515809683558349311967871741374280204310640456813777101114418102469710564429830158975054516139878640672147529108311238304126949020660912076693315472146673006153299957739385254809709800083518496038331097517102005680446426288033447441291159382214166319904520980934968838697095308970985819552768556648828878106906443304272298213265536617657160946184055678470292583785873756721657006774396623077812976685126075432759111923818350084951709482257536199045933991001590153033139356999719204891490277097958737963532711296206733861229816377635826501868691027845316178775114282168258752818907493506737083699050819888741401957399163005363413645041406534941740425657334727284920899933137087672220352650598915747859596622222132693412824846594121852407527765881750698162673158333417473302984075493762952473586696760031065919045848143247249729897628345891058544943945346718119748990257959771372571019238142477513494327226646524295181713189172679155743864157944494837802805710166814570717670953960055470479517987396639303800444867582810507965832868620443448443416402528272284255675761909170785205247962390717259012459695010325609323996182002501647136422612090836375388025367081687048850580605106876261516179043100537298590103937475358841322610616229911092141766693138914894129950152489520731361768769793073759677380743211963329112820469667737786144860530823378081628657785696856247689539634069418432366721198324931707551580851540315264238655213314100305796296183739296485666357575955800991297641237002504158971972294371528601202388802986384615612874778637624104472953394013652299354146384793495429780258463386039807390159663332688638810033538343552872531388650021729635520682351418966244852670567850618106181792164667512515552076128086969039451256790353195427834789631156662610761112141906619810743692519734100478945156813566464791331418284794567490084364543227088644479948161459382305910638570343827321644873196649631333118803773089378551377118310117559977652172445194685229031654420985923523841882644806327887470907910550110165846686847931672356389360175373726088891337857942051139827081883464327837347175077037573099038440770588883365096242868424907734594628515730655963251045539945835870561680571361562334647322862147546560418340863849470927665775655527162454287886882456450749382919278568904511680159304515925599919723018752674797266371687307702593280077498356015134265847123434626543584651150865724045628100039014713568158334391725231296131985263154555012891083439548122346407325448974031938421373752171937098998746080439319019997243649267865801377592283722141394117314784812986485061038586455642694140109134125112855980205701142141551831127980317224759393250971933507952594892822299580188781651281647183642682830244458624557885031189955978557582991079258031144169466314191250858675450250439183300945549985673804287565590544002566931933089952241674661171191947305397107726768101759323253298759965416179116677137076479896417380004236261804584011690236407562817576189450269265298740159793818915080113235405366865227173400847249734814248090808697174711860508719665850383074480344511696974930127688594547925246165284032157375093131763132737153973504693890803562875512670067484692416823272471844676781184244930124794292106091115177658946870572
n = 123520409794027950149082272849261810210272860945985519621068227141432450531116939513048892636595227082104407543359822151513351465688818819401575805963746384584750564932313178949378125144839419075875084826644611606705240992136187882428274030275790017773716142653357850577727089079538275883143754223406966004737
e = 65537
enc_0 =    76012148419596161794804416386132136699427747757658687602983691352043217047592310323871026989774905956383650384122156442537625793199561240152381187578650102568660967381854393349807030682800181738709716041263953681233180024763392338534720304399232267688344153695144303340694410117780223979389987971110632730679
enc_01 =   7601214841959616179480441638613213669942774775765868760298369135204321704759231032387102698977490595638365038412215644253762579319956124015238118757865010256866096738185439334980703068280018173870971604126395368123318002476339233853472030439923226768834415369514430334069441011778022397938998797111063273067986356293073038841959962480789553830481472509986470134456604526155419920420545907778777722830845304855462260975393065239838869739451771249056014945397142629798801847947095824094618775031683520183883070749435851492055428719292650660035774239111818180327314861430208309796627072625367415147638695534122781216168
enc_01_2 = 8635629307303884195996248078955383048147250998647013445660452615541992042054590777877772283084530485546226097539306523983886973945177124905601494539714262979880184794709582409461877503168352018388307074943585149205542871929265066003577423911181818032731486143020830979662707262536741514763869553412278121616876012148419596161794804416386132136699427747757658687602983691352043217047592310323871026989774905956383650384122156442537625793199561240152381187578650102568660967381854393349807030682800181738709716041263953681233180024763392338534720304399232267688344153695144303340694410117780223979389987971110632730679

In [4]:
# picoCTF{

In [7]:
# changing this per character 
from pwnlib.tubes.remote import remote
from string import ascii_letters
import re
r = remote('mercury.picoctf.net', 4572)
BIG_RECV = 9999999
temp = r.recv(BIG_RECV).decode()
print(temp)
flag = int(temp.split(": ")[1])
temp = r.recv(BIG_RECV).decode()
print(temp)
n = int(temp.split(": ")[1])
e = 65537
encs = []
def encrypt(s: str):
    r.recv(BIG_RECV, timeout=2)
    r.send(s.encode()+b"\n")
    temp = re.sub(r"[^0-9]", "", r.recv(BIG_RECV).decode())
    return temp

temp_a = []
for i in range(1,21):
    print(i)
    temp_a.append(encrypt(i*"a"))

for c in ascii_letters+"0123456789_":
    encs.append(encrypt("aaaaaaaa"+c))
    print(c)
# question(True, 4636878989)
# question(True, 93089)
# question(False)
# question(True, 836623060)
# question(True, 256931246631782714357241556582441991993437399854161372646318659020994329843524306570818293602492485385337029697819837182169818816821461486018802894936801257629375428544752970630870631166355711254848465862207765051226282541748174535990314552471546936536330397892907207943448897073772015986097770443616540466471245438117157152783246654401668267323136450122287983612851171545784168132230208726238881861407976917850248110805724300421712827401063963117423718797887144760360749619552577176382615108244813)
# question(False)
# question(True, 1405046269503207469140791548403639533127416416214210694972085079171787580463776820425965898174272870486015739516125786182821637006600742140682552321645503743280670839819078749092730110549881891271317396450158021688253989767145578723458252769465545504142139663476747479225923933192421405464414574786272963741656223941750084051228611576708609346787101088759062724389874160693008783334605903142528824559223515203978707969795087506678894006628296743079886244349469131831225757926844843554897638786146036869572653204735650843186722732736888918789379054050122205253165705085538743651258400390580971043144644984654914856729)
# question(True, 14311663942709674867122208214901970650496788151239520971623411712977119660454037678408741501)
# print(r.recv(BIG_RECV).decode())


flag: 8545896617061853564240414376367612565215768636576754123018898627147254268658838062955245117310577839487739642647993469099374315333562140969921564837775000849187365269018148347285386537691092707408908680869165177827226761313318724961696466173523361453493431374422688196322400645243873979887200355960483131388049379021575563755125405761362385061946371420546047208947934248162666569526548219315195028378007636786245866050252830042428331951644614619340403737016212001837250214324153684626751186221464022087561376821078775947645241642523156355595028592069747152398536622442865312469805196451527279014217214059836066448430117605609967507004376447944037152479224710509675399597874291257717231259770832068449011939994826866080287813967455619610876656584249307459123143488597076306621493889383300824303655826756684204696252769028964815863615157720582361470237110735168668755506138347446021044971109303436991298237526802064849216492911012662929066393184483120381925250541865114379904971866504682043739829

847795273120620850431262101153453183606106016450756170745355063374807456391355342031962557333894020037698679779943884954468192122372249128369227443562972720781615550619911976959234704967220310072560021642175065664625867659251384620034560184080862727256946644288840291175893905032442783412721745399280310041609136767800909103808343888904050192414715447593221303510974387664675754461804124346512751854589661706185356424041437636761256521576863657675412301889862714725426865883806295524511731057783948773964264799346909937431533356214096992156483777500084918736526901814834728538653769109270740890868086916517782722676131331872496169646617352336145349343137442268819632240064524387397988720035596048313138804457472646853239230036833028565657489314582709578688239300343047017855171422658087551032408398698918314291476462471066102856900145432124196384225904631214297673861303856763318439446417441422298221296477161102201721945850422647005538589212630482283228918988095914964261552333593390818317993244595732554765996741362791711948537994368465647569834959589731929830761075271325917005196151855713280681142513714052036439693716024903166376441898079413036590010218900603327836504535878413829993880042220533788026468192572450636163955880132201287795552477144195086324527035738143633720465105371972266096943808594306652698353604
647283280281098639263339955221066282168342669865036546850887869966495905262180403489722990510245977698878732608873511305678253968184433325682579289721085253747174007774481148088510804660302614407413401649735623350373773240092762112314068208817458633627004032509090529006343368088798479666308694718306923231413172607467183061675662917428319811233801063251518025625869885842563912183606084057907079903645608351363045518968334954850458910171363944349171337012284183022784342534849897690725094988706046438104311091312080572993048336975056993294820962615326381191153855088265586044010035592186280969351272050436540140645945128763243191891173214780960988003020535512525685116425574269747496408883262239215333085534366364991981602349909469865841355739038050101864808655215862498076898936471527797578040437394782270405512913788395169811876300410349925927048613040709616235881648799626010597819256124831551879061925637790477019639949854383041383495978850472741697853293972508231859121233009190736164981496304932638216584532326086004300200084543711647178006833627449248242048134672721478204642941995327178888699057701674487696419515487831449040495391485084867710633133537446073907326051289589090397709854481812778742826663711173658900175999467566369226221178706619413275093800239774941556320264410338488054192672271946025986151632105233754732794791316272509530734734497228677295785086408014333046397518953674904627155477564944984971342104853293583135829408758201430862977229004425230271166372667148617017138542389751021972093044507975415408595360455445747264685323923003683302856565748931458270957868823930034304701785517142265808755103240839869891831429147646247106610285690014543212419638422590463121429767386130385676331843944641744142229822129647716110220172194585042264700553858921263048228322891898809591496426155233359339081831799324459573255476599674136279171194853799436846564756983495958973192983076107527132591700519615185571328068114251371405203643969371602490316637644189807941303659001021890060332783650453587841382999388004222053378802646819257245063616395588013220128779555247714419508632452703573814363372046510537197226609694380859430665269835360485458966170618535642404143763676125652157686365767541230188986271472542686588380629552451173105778394877396426479934690993743153335621409699215648377750008491873652690181483472853865376910927074089086808691651778272267613133187249616964661735233614534934313744226881963224006452438739798872003559604831313880367443242471020421133199107478004037283444129704030869573557245273624009353781704448393971766447865009448261543062262489589422060329321536252077111148364284377474261965752664868840091786074546784280009806854114249622229620015158279972767232479044194767830646347333464989351356397767397696141382771255989075641102460019524074336656995094671469186905019053043292590647377795357478912620535678724736115487734499071700778709554223194479312379122033146333482789443970793503953404189273202751460480108647069301034565972205766686186947926917885910629445873204581229467142191847014534629803733590941924486662469170828503952757333894020037698679779943884954468192122372249128369227443562972720781615550619911976959234704967220310072560021642175065664625867659251384620034560184080862727256946644288840291175893905032442783412721745399280310041609136767800909103808343888904050192414715447593221303510974387664675754461804124346512751

In [9]:
temp_a[0]

'31726074671830616756629174283198112338010632515180256258698858425639121836060840579070799036456083513630455189683349548504589101713639443491713370122841830227843425348498976907250949887060464381043110913120805729930483369750569932948209626153263811911538550882655860440100355921862809693512720504365401406459'

In [15]:
[len(c) for c in temp_a]

[308,
 616,
 924,
 1232,
 1540,
 1848,
 2156,
 2463,
 2771,
 3079,
 3387,
 3695,
 4003,
 2811,
 0,
 140,
 574,
 4083,
 4045,
 1320]

In [62]:
[str(i) in str(flag) for i in [int(str(c).replace(str(temp_a), "")) for c in encs]]

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

In [55]:
ascii_lowercase[[str(i) in str(flag) for i in [int(str(c).replace(str(temp_a), "")) for c in encs]].index(True)]

'i'

In [8]:
len(str(pow(341431434431, e , n)))

308

In [57]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_v1_5, PKCS1_OAEP
# cipher = PKCS1_v1_5.new(RSA.construct((n, e)))
bytes_to_long(bin(bytes_to_long(b"0"))[2:].encode())

54087331622960

In [59]:
pow(54087331622960, e, n)

81767812622294119578952900203250807275608183046407693746308652407599261068649645126774773647206257258987580708576680983475179383258378115267267997417353095592024396549658616835562730658131841506386871543859069652735359606488860988281180174126973478424623632307849301998130639042666004982777558018864671615466

In [62]:
84322585877660588211630434818509514288162993861316341781209943167725485264862734160566627032810428274811481538953945957486349247974819382835655451272171813100061766379320113434517355735597851090595262830841381852987737353129010758878983937946640807411204348729514704788409176178404914875716589789499853153444==84322585877660588211630434818509514288162993861316341781209943167725485264862734160566627032810428274811481538953945957486349247974819382835655451272171813100061766379320113434517355735597851090595262830841381852987737353129010758878983937946640807411204348729514704788409176178404914875716589789499853153444

True

In [ ]:
# working solution
# technically can find flag in 1 run, but crash occurred (connected for too long or sth)
from pwnlib.tubes.remote import remote
from string import ascii_letters, ascii_lowercase, ascii_uppercase
import re
r = remote('mercury.picoctf.net', 4572)
flag = r.recvuntil(b"n: ").decode()
n = r.recvuntil(b"e: ").decode()
e = 65537
r.recvuntil([b"\nI will encrypt whatever you give me: ", b"I will encrypt whatever you give me: "])
# print(r.recvuntil([b"\nI will encrypt whatever you give me: ", b"I will encrypt whatever you give me: "]).decode())
def encrypt(s: str):
    r.send(s.encode()+b"\n")
    temp = re.sub(r"[^0-9]", "", r.recvuntil([b"\nI will encrypt whatever you give me: ", b"I will encrypt whatever you give me: "]).decode())
    return temp
mask = []
for i in range(1,40):
    print(i, end="")
    mask.append(encrypt(i*"%"))
print()
seg_masks = []
for i in range(len(mask)):
    m = mask[i]
    for sm in seg_masks:
        m = m.replace(sm, "")
    seg_masks.append(m)
ascii_dir = ascii_lowercase+"0123456789_"+ascii_uppercase
flag_out = ""
for i in range(20,40):
    encs = []
    for c in ascii_dir:
        enc = encrypt(i*"%"+c)
        for sm in seg_masks:
            enc = enc.replace(sm, "")
        encs.append(enc)
        print(c, end="")
        if enc in flag: break
    truths = [c in flag for c in encs]
    print()
    if True in truths:
        char_flag = ascii_dir[truths.index(True)]
        flag_out+=char_flag
        print(char_flag)
    else:
        print("idk")
print(flag_out)

In [26]:
len("picoCTF{bad_1d3a5_3525501}")

20